In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [2]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


video_path =  configParser.get('test_imagen','video_path')

ttwbdf =  int(configParser.get('test_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [3]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [4]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [5]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [6]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('test_imagen', 'audio_length_used') 
model_filename =  configParser.get('test_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('test_imagen', 'openl3_mode')
folder =  configParser.get('test_imagen', 'folder')
number_of_images =  configParser.get('test_imagen', 'number_of_images')

In [7]:
age =  int(configParser.get('test_imagen', 'age'))
ethnicity =  str(configParser.get('test_imagen', 'ethnicity'))
gender =  str(configParser.get('test_imagen', 'gender'))
language =  str(configParser.get('test_imagen', 'language'))

In [8]:
dataGotten = [['1',video_path,age,ethnicity,gender,[],[],language,[]]]
df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','AGE','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path'])


In [9]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#  tf.config.experimental.set_memory_growth(gpu, True)

2023-07-08 21:57:09.580038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-08 21:57:09.603974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-08 21:57:09.604502: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 21:57:10.140878: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
#gpus

[]

In [11]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
import multiprocessing

audio_embs =  configParser.get('COMMON', 'audio_embs') 
from pydub import AudioSegment
import math
# Get original duration of video
audio = AudioSegment.from_file(video_path)
audio_length_og = math.floor(audio.duration_seconds)


video_filename = os.path.basename(video_path)

absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


import subprocess
# Extract audio monochannel and with 16khz and put it in absPathAudio

command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
subprocess.call(command, shell=True)

from threading import Thread
import time

# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass


# Will either truncate or loop the original video to reach audio_length (6,12 or 24)
audio_length_list = [24]
for audio_length in audio_length_list:
    path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)




    q = multiprocessing.Queue()

    proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(q,path_var_len_audio,))
    proc.start()
    proc.join()



    df['SPEAKER_EMB'] = q.get()
    print(df['SPEAKER_EMB'].head())
    proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(q,path_var_len_audio,audio_embs,audio_length,openl3_mode,))
    proc.start()
    proc.join()

    df['AUDIO_EMB'] = q.get()
    print(df['SPEAKER_EMB'].head())

    

    ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
    tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
    tDelete.start()



[NeMo W 2023-07-08 21:57:13 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-07-08 21:57:13 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


0    b'\x80\x04\x95\x8f\x03\x00\x00\x00\x00\x00\x00...
Name: SPEAKER_EMB, dtype: object


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.0+rocm5. Bad things might happen unless you revert torch to 1.x.


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`


...

[NeMo W 2023-07-08 21:57:18 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-08 21:57:18 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-07-08 21:57:18 features:291] PADDING: 16
[NeMo I 2023-07-08 21:57:18 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.
[NeMo I 2023-07-08 21:57:18 cloud:58] Found existing object /home/gamal/.cache/torch/NeMo/NeMo_1.19.0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-07-08 21:57:18 cloud:64] Re-using file from: /home/gamal/.cache/torch/NeMo/NeMo_1.19.0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-07-08 21:57:18 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-08 21:57:18 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-07-08 21:57:18 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-07-08 21:57:18 features:291] PADDING: 16
[NeMo I 2023-07-08 21:57:19 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/torch/NeMo/NeMo_1.19.0/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.


Process Process-2:
Traceback (most recent call last):
  File "/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gamal/vsc/DiffusionSpeech2Face/testing_imagen_pyannote_titanet.py", line 118, in extract_pyannote_titanet_embeddings
    emb0 = inference0(path_var_len_audio).data
  File "/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/pyannote/audio/core/inference.py", line 425, in __call__
    return self.slide(waveform, sample_rate, hook=hook)
  File "/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/pyannote/audio/core/inference.py", line 275, in slide
    self.model.example_output,
  File "/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/functools.py", line 967, in __get__
    val = self.func(instance)
  File "/home/gamal/anacon

In [ ]:
df3 = df[["image_path","caption_e","caption_g","caption_l"]]
data_frame = df3
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']

In [ ]:
def speaker_emb_preprocess(speaker_emb2):

    speaker_emb2 = pickle.loads(speaker_emb2)
    speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    speaker_emb2 = speaker_emb2 / 200.0
    speaker_emb2 = np.array(speaker_emb2).tolist()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(250-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    arr = arr / (1 if(audio_embs == 'wav2vec') else 10 if(audio_embs == 'openl3')  else 1 if(audio_embs == 'pyannoteTitaNet') else 1)
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

from PIL import Image
def getImage(face_path):
    im = Image.open(face_path)
    im.load() # required for png.split()

    im2 = Image.new("RGB", im.size, (255, 255, 255))
    im2.paste(im, mask=im.split()[3]) # 3 is the alpha channel
    im3 = np.array(im2)
    im4 = np.rollaxis(im3,2)
    return im4

In [ ]:
import random


def process_age(age):
    if(age > 100 or age < 0):
        x = np.zeros(768)
        x[767] = 1
        return x
    else:
        x = np.ones(768) * (age / 100.0)
        x[767] = 0
        return x

def process_gender(gender):
    if(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x        

In [ ]:
def process_input(age,ethnicity,gender,language,speaker_emb,audio_emb):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((age, ethnicity))
    h = np.vstack((h, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))

    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()

from PIL import Image
def process_image_path(path):
    
    return path

In [ ]:
data_frame['SPEAKER_EMB'] = data_frame['SPEAKER_EMB'].apply(lambda x: speaker_emb_preprocess(x))
data_frame['AUDIO_EMB'] = data_frame['AUDIO_EMB'].apply(lambda x: audio_emb_preprocess2(x))
data_frame['caption_a'] = data_frame['caption_a'].apply(lambda x: process_age(x))
data_frame['caption_g'] = data_frame['caption_g'].apply(lambda x: process_gender(x))
data_frame['caption_l'] = data_frame['caption_l'].apply(lambda x: process_language(x))
data_frame['caption_e'] = data_frame['caption_e'].apply(lambda x: process_ethnicity(x))

data_frame['INPUT'] = data_frame['SPEAKER_EMB']

for index, row in data_frame.iterrows():
    x = process_input(data_frame.loc[index,"caption_a"],data_frame.loc[index,"caption_e"]
                        ,data_frame.loc[index,"caption_g"],
                    data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                    data_frame.loc[index,"AUDIO_EMB"])
    x = [x]
    #AADFS = AADFS
    data_frame.loc[index,"INPUT"] = x

data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)


data_frame['image_path'] = data_frame['image_path'].apply(lambda x: process_image_path(x))

input = data_frame['INPUT'].to_numpy()
input = np.array([np.array(xi) for xi in input])
input[np.isnan(input)] = 0
input[input > 10] = 10
input[input < -10] = -10

output = data_frame['image_path'].to_numpy()
output = np.array([np.array(xi) for xi in output])
output.squeeze().shape


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

input = torch.from_numpy(input)
input = input.to(torch.float)

output = torch.from_numpy(output)
output = output.to(torch.float)

input = input.squeeze()
output = output.squeeze()


In [ ]:
input = input[None,:,:]

In [ ]:
unet_dim =  int(configParser.get('COMMON', 'unet_dim'))


In [ ]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset

unet1 = Unet(
    dim = unet_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = unet_dim,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

#unet = Unet(
#    dim = 32,
#    dim_mults = (1, 2, 4, 8),
#    num_resnet_blocks = 1,
#    layer_attns = (False, False, False, True),
#    layer_cross_attns = False
#)

# imagen, which contains the unet above

#imagen = Imagen(
#    unets = unet,
#    image_sizes = 32,
#    timesteps = 1000
#)

imagen = Imagen(
    unets = (unet1, unet2),
    image_sizes = (image_size, image_size),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()

print('Using model file ' + model_filename)
trainer.load(model_filename)





In [ ]:

if not os.path.exists(folder):
    os.makedirs(folder)

for i in range(number_of_images):
    images = trainer.sample(text_embeds=input,stop_at_unet_number=1,batch_size = 1, return_pil_images = True) # returns List[Image]
    t = time.time()
    images[0].save(folder + '/' + str(t) + '.png')


In [ ]:
import os
os._exit(0)